In [ ]:
# packages
import os
import sys
import scipy
import scipy.io as sio
import scanpy.external as sce
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import umap.umap_ as umap
import re
import anndata
import scirpy as ir
from matplotlib.gridspec import GridSpec
import random
import scipy.stats as stats
from scipy.stats import median_abs_deviation

In [ ]:
# read in

# The DGE_filtered folder contains the expression matrix, genes, and files [QCed in Parse pipeline before export]
adata = sc.read_mtx("count_matrix.mtx")

# reading in gene and cell data
carotid_gene_data = pd.read_csv("all_genes.csv")
carotid_cell_meta = pd.read_csv("cell_metadata.csv")

# find genes with nan values and filter
carotid_gene_data = carotid_gene_data[carotid_gene_data.gene_name.notnull()]
notNa = carotid_gene_data.index
notNa = notNa.to_list()

# remove genes with nan values and assign gene names
adata = adata[:,notNa]
adata.var = carotid_gene_data
adata.var.set_index('gene_name', inplace=True)
adata.var.index.name = None
adata.var_names_make_unique()

# add cell meta data to anndata object
adata.obs = carotid_cell_meta
adata.obs.set_index('bc_wells', inplace=True)
adata.obs.index.name = None
adata.obs_names_make_unique()

#basic qc
sc.pp.filter_cells(adata, min_counts=100)
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=5)
adata.shape

In [ ]:
adata.var_names_make_unique()

In [ ]:
# based on www.sc-best-practices.org/preprocessing_visualization/quality_control.html

# thorough add'l qc

# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)

p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
# sc.pl.violin(adata, 'total_counts')
p2 = sc.pl.violin(adata, "pct_counts_mt")
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
)

In [ ]:
# filter cells based on qc


adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 5) | (
    adata.obs["pct_counts_mt"] > 10
)
adata.obs.mt_outlier.value_counts()

print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

In [ ]:
# normalize
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
# highly var
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.25)
sc.pl.highly_variable_genes(adata, save='') # scanpy generates the filename automatically

# Save raw expression values before variable gene subset
adata.raw = adata

# remove IR genes from highly var
for i in adata.var.index:
    if re.search('^IG[HKL][VDJC]', i):
        adata.var.at[i, 'highly_variable'] = False

for i in adata.var.index:
    if re.search('^IG[HKL][VDJ]|IGHM|IGHD|IGHE|IGHA[1-2]|IGHG[1-4]|IGKC|IGLC[1-7]|AC233755.1|TR[ABGD][CV]|IGLL', i):
        adata.var.at[i, 'highly_variable'] = False       

adata = adata[:, adata.var.highly_variable]

sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, svd_solver="arpack")
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50, save="")

In [ ]:
# harmonize
sce.pp.harmony_integrate(adata, 'sample', random_state=65)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30, use_rep="X_pca_harmony")
sc.tl.umap(adata, random_state=68)
sc.tl.leiden(adata, resolution=0.7, random_state=68)
sc.pl.umap(adata, color=['leiden', 'sample'], legend_fontsize=8)

In [ ]:
sc.tl.rank_genes_groups(adata, "leiden", method="wilcoxon")
sc.pl.rank_genes_groups(adata, n_genes=15, sharey=False)
top_markers = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)
print(top_markers)

In [ ]:
sc.external.pp.scrublet(adata, sim_doublet_ratio=2, expected_doublet_rate=0.05, stdev_doublet_rate=0.02, batch_key=None, random_state=42)